# Multiple Linear Regression
God i hope this works

In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import r2_score, max_error, mean_absolute_error, mean_squared_error
from dataset_storing import cal_fire_temps, k_fold, fire_X, fire_centroid_x, fire_centroid_y, fire_area
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer

this is working


In [ ]:
cal_fire_temps.head(1)

,10_MONTHS_BEFORE,11_MONTHS_BEFORE,12_MONTHS_BEFORE,1_MONTHS_BEFORE,2_MONTHS_BEFORE,3_MONTHS_BEFORE,4_MONTHS_BEFORE,5_MONTHS_BEFORE,6_MONTHS_BEFORE,7_MONTHS_BEFORE,...,9_MONTHS_BEFORE,ALARM_DATE,CAUSE,GIS_ACRES,Shape_Area,Shape_Length,YEAR_,centroid_x,centroid_y,geometry
489,"[20.307, 20.325, 26.785, 20.325, 25.265, 20.30...","[21.683000000000003, 20.601, 26.483, 20.601, 2...","[19.936, 18.781, 21.254, 18.781, 20.896, 19.93...","[17.88, 15.861999999999998, 15.863, 15.8619999...","[17.609, 12.479, 11.754, 12.479, 14.687, 17.60...","[14.373, 9.202, 7.236000000000001, 9.202, 10.0...","[12.751, 8.165, 5.541, 8.165, 8.391, 12.751, 8...","[13.76, 10.718, 9.472, 10.718, 10.897, 13.76, ...","[15.414, 14.002, 14.239, 14.002, 14.809, 15.41...","[14.753, 14.117, 15.045, 14.117, 14.889, 14.75...",...,"[17.834, 18.829, 23.84800000000001, 18.829, 21...",2007-10-21,14.0,25.736713,104152.781054,1902.439051,2007,138213.432525,-402730.00539,"MULTIPOLYGON (((138036.906 -402646.363, 138086..."


## Predicting Area

In [3]:
split_indices = k_fold.split(fire_X, fire_centroid_x)
imputer = SimpleImputer()
fire_X = imputer.fit_transform(fire_X)

accuracy_area_dict = {
    "mean_error_train": [],
    "mean_error_test": [],
    "r2_score_train": [],
    "r2_score_test": []
}

for i, (train, test) in enumerate(split_indices):
    # flattening the lists
    train_X = fire_X[train]
    train_y = fire_area.iloc[train]
    test_X = fire_X[test]
    test_y = fire_area.iloc[test]

    model = LinearRegression()
    model.fit(train_X, train_y)
    
    print(f"Trial {i}:")
    y_pred = model.predict(test_X)
    y_pred_train = model.predict(train_X)
    print(r2_score(test_y, y_pred))
    
    accuracy_area_dict["mean_error_train"].append(np.float128(mean_absolute_error(train_y, y_pred_train)))
    accuracy_area_dict["mean_error_test"].append(np.float128(mean_absolute_error(test_y, y_pred)))
    accuracy_area_dict["r2_score_train"].append(np.float128(r2_score(train_y, y_pred_train)))
    accuracy_area_dict["r2_score_test"].append(np.float128(max_error(test_y, y_pred)))


Trial 0:
0.003171409376149792
Trial 1:
-1.518335331081101e+18
Trial 2:
-0.006246502424815503
Trial 3:
0.007490289190595578
Trial 4:
-0.028284681235324127
Trial 5:
-3.6584047529136916e+18
Trial 6:
0.003588850730800397
Trial 7:
-0.011735678285131224
Trial 8:
0.004515950942555191
Trial 9:
0.01356474114094075
Trial 10:
-0.007249120611656279
Trial 11:
-0.047001180629647754
Trial 12:
-1.1594741270320366e+18
Trial 13:
0.005784315106862725
Trial 14:
-0.042700451471261225
Trial 15:
0.006190832530666879
Trial 16:
0.0021750931742016144
Trial 17:
-1.1708336771316436e+18
Trial 18:
-0.009732862523778696
Trial 19:
0.0031488917977345876
Trial 20:
-0.04644783847844258
Trial 21:
-0.01580909027401578
Trial 22:
-0.017251820007198715
Trial 23:
0.010715030776220491
Trial 24:
-8.851081918881874e+18


In [ ]:
accuracy_area_df = pd.DataFrame(accuracy_area_dict, index=range(1, 26))
accuracy_area_df.head()

accuracy_area_df.to_csv("data/mlr_area.csv")

## Predicting X and Y coords?

In [ ]:
split_indices = k_fold.split(fire_X, fire_centroid_x)

accuracy_x_dict = {
    "mean_error_train": [],
    "mean_error_test": [],
    "r2_score_train": [],
    "r2_score_test": []
}

for i, (train, test) in enumerate(split_indices):
    # flattening the lists
    train_X = fire_X[train]
    train_y = fire_centroid_x.iloc[train]
    test_X = fire_X[test]
    test_y = fire_centroid_x.iloc[test]

    model = LinearRegression()
    model.fit(train_X, train_y)
    
    print(f"Trial {i}:")
    y_pred = model.predict(test_X)
    y_pred_train = model.predict(train_X)
    print(r2_score(test_y, y_pred))
    
    accuracy_area_dict["mean_error_train"].append(np.float128(mean_absolute_error(train_y, y_pred_train)))
    accuracy_area_dict["mean_error_test"].append(np.float128(mean_absolute_error(test_y, y_pred)))
    accuracy_area_dict["r2_score_train"].append(np.float128(r2_score(train_y, y_pred_train)))
    accuracy_area_dict["r2_score_test"].append(np.float128(max_error(test_y, y_pred)))


Trial 0:
495960.42574048863
33080028075.414402
154323.59412344932
0.0703471828475658
Trial 1:
1.0741330548563164e+16
4.3686551288714975e+28
4067145380374.35
-1.235314713033393e+18
Trial 2:
528227.6510833949
34300317567.287655
156845.68848678222
0.06610676145073646
Trial 3:
570809.5312168945
33377638475.74658
154178.32542776846
0.06798042185580466
Trial 4:
555671.6933386201
34588869257.686485
157175.8335233777
0.07531654330322046
Trial 5:
9.7195603793528e+16
3.5770486167317206e+30
36802576375493.3
-9.694384530885639e+19
Trial 6:
468124.1336762459
32907957066.056026
152837.41278654808
0.07715264016326662
Trial 7:
546005.6173644605
34368878237.57479
156406.3412825894
0.06102973649388399
Trial 8:
532585.956363338
33190997262.970264
152767.5896391477
0.07444051482315073
Trial 9:
537685.8496584429
33993226041.896923
156311.15018360678
0.054135357158841324
Trial 10:
547600.0563048883
33185319598.97037
154116.65457217387
0.06396166401101722
Trial 11:
533341.1074475292
34110986166.559258
156338

In [ ]:
accuracy_x_df = pd.DataFrame(accuracy_x_dict, index=range(1, 26))
accuracy_x_df.to_csv("data/mlr_cent_x.csv")

Predicting y

In [ ]:
split_indices = k_fold.split(fire_X, fire_centroid_x)

accuracy_y_dict = {
    "mean_error_train": [],
    "mean_error_test": [],
    "r2_score_train": [],
    "r2_score_test": []
}

for i, (train, test) in enumerate(split_indices):
    # flattening the lists
    train_X = fire_X[train]
    train_y = fire_centroid_y.iloc[train]
    test_X = fire_X[test]
    test_y = fire_centroid_y.iloc[test]

    model = LinearRegression()
    model.fit(train_X, train_y)
    
    print(f"Trial {i}:")
    y_pred = model.predict(test_X)
    y_pred_train = model.predict(train_X)
    print(r2_score(test_y, y_pred))
    
    accuracy_area_dict["mean_error_train"].append(np.float128(mean_absolute_error(train_y, y_pred_train)))
    accuracy_area_dict["mean_error_test"].append(np.float128(mean_absolute_error(test_y, y_pred)))
    accuracy_area_dict["r2_score_train"].append(np.float128(r2_score(train_y, y_pred_train)))
    accuracy_area_dict["r2_score_test"].append(np.float128(max_error(test_y, y_pred)))


Trial 0:
755067.3342216053
2.3517976776515455
263772.8970352781
0.07165278629787986
Trial 1:
3.478811733193447e+16
84612459.0469842
13172327910964.188
-4.583848714774355e+18
Trial 2:
821746.8349938565
3.2922099238594726
259921.53883153308
0.0827907526184336
Trial 3:
720441.7749514058
3.2239067172325617
258293.9981253333
0.08037267323846664
Trial 4:
736125.6444978192
2.7616118709091566
258165.2839836952
0.08311458334786603
Trial 5:
1.4887796270800486e+17
362104384.79793024
56371814995031.73
-8.350784273292013e+19
Trial 6:
720036.4860326496
2.3523193713682193
258780.01338466845
0.09337767838225375
Trial 7:
769815.3869787185
2.777244734257443
259950.08716975813
0.07503845856420488
Trial 8:
791829.420293221
4.125170372720476
256437.81040698063
0.09625595387182317
Trial 9:
735309.7765150141
3.347080477327942
260043.95814950674
0.05703406203340522
Trial 10:
740710.4647122878
5.028551006604684
257404.86429870295
0.07888867536276278
Trial 11:
845923.0750457047
2.8328862882280403
260563.9635061

In [ ]:
accuracy_y_df = pd.DataFrame(accuracy_y_dict, index=range(1, 26))
accuracy_y_df.to_csv("data/mlr_cent_y.csv")